In [1]:
import os
import shutil
import pandas as pd

# Shuttle WRR In Process PDFs from "old" ticketid folder structure to "new" folder structure
## 24.11.06
## Steps:
1. Associate old TIDs to New TIDs
2. Locate folders/pdfs based on old TIDs
3. Transfer folders/pdfs to new TIDs
    - How to manage aggregation where it didn't exist before? (multiple week 1s for example)
4. Upload to G-drive

In [3]:
# Transfer function from the WAR project
# WAR didn't have any new aggregations of activities already in process.
# WRR does - so I expect to have issues of multiple week labels

def copy_pdfs_to_temp(src_directory, temp_directory):
    os.makedirs(temp_directory, exist_ok=True)
    
    for week_folder in os.listdir(src_directory):
        week_path = os.path.join(src_directory, week_folder)
        
        if os.path.isdir(week_path) and week_folder.startswith("Week"):
            new_week_path = os.path.join(temp_directory, week_folder)
            os.makedirs(new_week_path)
            
            for file_name in os.listdir(week_path):
                if file_name.lower().endswith("pdf"):
                    file_path = os.path.join(week_path, file_name)
                    shutil.copy(file_path, new_week_path)
                    print(f"Copied: {file_path} to {new_week_path}")